# SageMaker Builtin - K-Means - Example

Original example:

https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_applying_machine_learning/US-census_population_segmentation_PCA_Kmeans/sagemaker-countycensusclustering.ipynb

### Analyze US census data for population segmentation using Amazon SageMaker¶


This is modified to run from a local SageMaker instance.


## Step 1: Load the Data

In [ ]:
# relevant libraries

import os
import boto3
import io
import sagemaker

%matplotlib inline 

import pandas as pd
import numpy as np

# import mxnet as mx
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
matplotlib.style.use('ggplot')
import pickle, gzip, urllib, json
import csv

In [ ]:
!which python


In [ ]:
! which pip

In [ ]:
import mxnet as mx

In [ ]:
# execute this on aws sagemaker
from sagemaker import get_execution_role
role = get_execution_role()

In [ ]:
# execute this on local sagemaker
from sagemaker import get_execution_role

# use this if running sagemaker locally
def resolve_sm_role():
    client = boto3.client('iam', region_name='us-east-2')
    response_roles = client.list_roles(
        PathPrefix='/',
        # Marker='string',
        MaxItems=999
    )
    for role in response_roles['Roles']:
        if role['RoleName'].startswith('AmazonSageMaker-ExecutionRole-'):
            print('Resolved SageMaker IAM Role to: ' + str(role))
            return role['Arn']
    raise Exception('Could not resolve what should be the SageMaker role to be used')

# this is the role created by sagemaker notebook on aws
role_arn = resolve_sm_role()
print(role_arn)



In [ ]:
iam = boto3.client('iam')
role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20200201T085431')['Role']['Arn']
# or
role = role_arn
print(role)

In [ ]:
s3_client = boto3.client('s3')
data_bucket_name='aws-ml-blog-sagemaker-census-segmentation'

In [ ]:
obj_list=s3_client.list_objects(Bucket=data_bucket_name)
file=[]
for contents in obj_list['Contents']:
    file.append(contents['Key'])
print(file)

In [ ]:
file_data=file[0]
file_data

### read file content from S3

In [ ]:
response = s3_client.get_object(Bucket=data_bucket_name, Key=file_data)
response_body = response['Body'].read()
counties = pd.read_csv(io.BytesIO(response_body), header=0, delimiter=',', low_memory=False)
counties.head()

## Step 2: Exploratory Data Analysis (EDA) - Data Cleaning and Exploration


In [ ]:
counties.shape

In [ ]:
# remove rows with missing values
counties.dropna(inplace=True)
counties.shape

In [ ]:
# set state-county'' as the index
counties.index=counties['State'] + '-' + counties['County']

# remove columns
drop=['CensusId', 'State', 'County']
counties.drop(drop, axis=1, inplace=True)



In [ ]:
# set some pandas options to see more data
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
counties.head()

### Visualize the data

In [ ]:
import seaborn as sns

In [ ]:
for a in ['Professional', 'Service', 'Office']:
    ax=plt.subplots(figsize=(6,3))
    ax=sns.distplot(counties[a])
    title="Histogram of " + a
    ax.set_title(title, fontsize=12)
    plt.show()
                             

## Feature engineering

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()
counties_scaled=pd.DataFrame(scaler.fit_transform(counties))

# get the columns from initial data-frame
counties_scaled.columns = counties.columns

# get the index from the initial data-frame
counties_scaled.index=counties.index
counties_scaled.head()

In [ ]:
counties_scaled.describe()

## Step 3: Data Modeling

### Dimension reduction

In [ ]:
counties_scaled.shape

In [ ]:
role

In [ ]:
bucket='md-ml-labs-bucket'
from sagemaker import PCA
num_components=33

In [ ]:
# instantiate a principal component analysis sagemaker object
pca_SM = PCA(
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    output_path='s3://' + bucket + '/counties/',
    num_components=num_components)

In [ ]:
# convert all values explicitly to float32
train_data = counties_scaled.values.astype('float32')

In [ ]:
%%time

# launch the pca job in sagemaker
pca_SM.fit(pca_SM.record_set(train_data))

In [ ]:
job_name = 'pca-2020-02-01-17-43-51-344'
model_key = "counties/" + job_name + "/output/model.tar.gz"

In [ ]:
# download model results
boto3.resource('s3').Bucket(bucket).download_file(model_key, './data/model.tar.gz')

In [ ]:
# un-tar to the same folder
os.system('tar -zxvf ./data/model.tar.gz -C ./data/')

In [ ]:
# # this is unnecessary
# os.system('unzip -d ./data/ ./data/model_algo-1')

In [ ]:
import mxnet as mx

In [ ]:

# load model parameters, that is the output of the pca job
pca_model_params = mx.ndarray.load('./data/model_algo-1')

In [ ]:
s = pd.DataFrame(pca_model_params['s'].asnumpy())
v = pd.DataFrame(pca_model_params['v'].asnumpy())

In [ ]:
print(s.shape)
print(s)

In [ ]:
print(v.shape)

In [ ]:
v

In [ ]:
n_comp=6
s.iloc[-n_comp:,:]

In [ ]:
s.iloc[-n_comp:,:].apply(lambda x: x*x).sum()

In [ ]:
s.apply(lambda x: x*x).sum()

In [ ]:
# this number explain how much variance of the total variance, is explained by the last n_comp components
s.iloc[-n_comp:,:].apply(lambda x: x*x).sum()/s.apply(lambda x: x*x).sum()

In [ ]:
# we'll take n_comp components from our components matrix
s_5 = s.iloc[-5:, :]
s_5

In [ ]:
v_5 = v.iloc[:, -5:]
v_5.columns = [0, 1, 2, 3, 4]
v_5

In [ ]:
# show the first component that 
comp_num = 1
first_comp = v_5[5 - comp_num]
print(first_comp)

In [ ]:
list(zip(first_comp, counties_scaled.columns))

In [ ]:
comps = pd.DataFrame(list(zip(first_comp, counties_scaled.columns)), columns=['weights', 'features'])
comps

In [ ]:
ax = sns.barplot(
#     data=comps.sort_values('abs_weights', ascending=False).head(10), 
    data=comps,
    x="weights", 
    y="features", 
    palette="Blues_d")
ax.set_title("PCA Component Makeup: #" + str(comp_num))
plt.show()

In [ ]:
PCA_list=['comp_1', 'comp_2', 'comp_3', 'comp_4', 'comp_5']

### Deploy

In [ ]:
%%time

# deploy
pca_predictor = pca_SM.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium')

In [ ]:
%%time

# predict
# apply original data to the pca predictor
result = pca_predictor.predict(train_data)

In [ ]:
counties_transformed=pd.DataFrame()

a1 = result[0]

In [ ]:
a1

In [ ]:
# loop over the result of the prediction and append each element to counties_transformed
for a in result:
    b=a.label['projection'].float32_tensor.values
    counties_transformed=counties_transformed.append([list(b)])

# get the index and columns from countries_scaled data-frame
# but keep the last 5 columns
counties_transformed.index=counties_scaled.index
counties_transformed=counties_transformed.iloc[:,-5:]
counties_transformed.columns=PCA_list

In [ ]:
counties_transformed.head()

### Use the transformed result with last 5 components to clasify counties using k-means

In [ ]:
from sagemaker import KMeans

In [ ]:
num_clusters = 7

# instantiate a KMeans object
kmeans = KMeans(
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    output_path='s3://'+ bucket +'/counties/',              
    k=num_clusters)

In [ ]:
# we replace the train_data with the results of pca reduction
train_data = counties_transformed.values.astype('float32')

In [ ]:
%%time

# run the KMeans clusterization 
kmeans.fit(kmeans.record_set(train_data))

In [ ]:
%%time

# deploy he nodel
kmeans_predictor = kmeans.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium')

In [ ]:
%%time

# predict on training data
result=kmeans_predictor.predict(train_data)

In [ ]:
# lets see how results looks like
a1 = result[0]
type(a1)

In [ ]:
a1

In [ ]:
# create an array out of the results
# this identify the closest cluster out of the 7 clusters that we run the fit for
# each county get a label
cluster_labels = [r.label['closest_cluster'].float32_tensor.values[0] for r in result]
cluster_labels[:10]


In [ ]:
# this counts the values ver each cluster out of the seven
# means how many counties belong to each cluster, by the distance to centroid 
pd.DataFrame(cluster_labels)[0].value_counts()

In [ ]:
ax = plt.subplots(figsize=(6,3))
ax = sns.distplot(cluster_labels, kde=False)

title="Histogram of Cluster Counts"
ax.set_title(title, fontsize=12)
plt.show()

## Step 4: Conclusions

### Access the KMeans model attributes

In [ ]:
job_name = 'kmeans-2020-02-01-19-17-27-635'
model_key = "counties/" + job_name + "/output/model.tar.gz"

# dowload the artifacts from s3 localy under "data-kmeans" folder
boto3.resource('s3').Bucket(bucket).download_file(model_key, './data-kmeans/model.tar.gz')
os.system('tar -zxvf ./data-kmeans/model.tar.gz -C ./data-kmeans/')

In [ ]:
os.system('unzip -d ./data-kmeans/ ./data-kmeans/model_algo-1 ')

In [ ]:
# load model attributes using mxnet
Kmeans_model_params = mx.ndarray.load('./data-kmeans/model_algo-1')

In [ ]:
# model parameters contains coordinates of the centroids in out 5 dimension space
# as an ndarray
Kmeans_model_params

In [ ]:
Kmeans_model_params[0]

In [ ]:
cluster_centroids=pd.DataFrame(Kmeans_model_params[0].asnumpy())
cluster_centroids.columns=counties_transformed.columns

In [ ]:
cluster_centroids

In [ ]:
plt.figure(figsize = (16, 6))
ax = sns.heatmap(cluster_centroids.T, cmap = 'YlGnBu')
ax.set_xlabel("Cluster")
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
ax.set_title("Attribute Value by Centroid")
plt.show()

In [ ]:
list(map(int, cluster_labels))[:10]

In [ ]:
# add a new column with labels representing the cluster they belong to
counties_transformed['labels']=list(map(int, cluster_labels))
counties_transformed.head()

## Cleanup

In [ ]:
sagemaker.Session().delete_endpoint(pca_predictor.endpoint)

In [ ]:
sagemaker.Session().delete_endpoint(kmeans_predictor.endpoint)